<p style="font-weight:bold;"> <span style="font-size: 36px"> Reports</span> </p>

<a id='report-production'></a>

# Infrastructure and Configuration

## Initialize data

In [ ]:
/* DataSource is configured and connected to real database */
//#!eval-notebook "../Database/Configure"

In [ ]:
#!eval-notebook "../Initialization/InitSystemorphToMemory"

In [ ]:
#!import "Ifrs17ReportScopes"

In [ ]:
Workspace.InitializeFrom(DataSource);

## Imports and Configurations

In [ ]:
var reportStorage = new ReportStorage(Workspace, Report);
await reportStorage.InitializeReportIndependentCacheAsync();
var mostRecentPartition = (await Workspace.Query<PartitionByReportingNodeAndPeriod>().Where(x => x.Scenario == null).OrderBy(x => x.Year).ThenBy(x => x.Month).ToArrayAsync()).Last();
var reportingNodeRoot = (await Workspace.Query<ReportingNode>().Where(x => x.Parent == null).ToArrayAsync()).First().SystemName;
await reportStorage.InitializeAsync((mostRecentPartition.Year, mostRecentPartition.Month), reportingNodeRoot, null, CurrencyType.Contractual);
var identities = reportStorage.GetIdentities((mostRecentPartition.Year, mostRecentPartition.Month), reportingNodeRoot, null, CurrencyType.Contractual);

## Calling Scope

In [ ]:
var universe = Scopes.ForSingleton().WithStorage(reportStorage).ToScope<IUniverse>();

# Report Settings and Storage Update

In [ ]:
((int Year, int Month) Period, string ReportingNode, string Scenario, CurrencyType CurrencyType) Args =
    //((2020, 12), reportingNodeRoot, null,CurrencyType.Contractual)
    ((2021, 3), reportingNodeRoot, null,CurrencyType.Contractual)
    //((2021, 3), reportingNodeRoot, null,CurrencyType.Functional)
    //((2021, 3), reportingNodeRoot, null,CurrencyType.Group)
     ;

await reportStorage.InitializeAsync(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);
identities = reportStorage.GetIdentities(Args.Period, Args.ReportingNode, Args.Scenario, Args.CurrencyType);

In [ ]:
var ifrs17 = new Ifrs17Reports(Workspace, Scopes, Report);

# Use cases

For demostration purposes we import data for 7 *Group of Insurance Contract* (GIC) and 4 *Group of Reinsurance Contract* (GRIC). 
<br> Data set consists in cashflows, actuals, and parameters.



# Present Value

Present values of the best estimate of future cashflows are shown here in an Analysis of Change report.

The report view can be modified with the Slice options for the columns by changing the <code>SliceColumnBy</code> inputs in the next command cell.
For example one can add <code>"GroupOfContract"</code> to separate the contributions of the individual Group of Contracts.
<br> We suggest to add this slice between the <code>"LiabilityType"</code> and the <code>"EconomicBasis"</code> as the the order of the inputs corresponds to the order of the columns shown in the report to expand the data.

Aggregated values are displayed when the data has a finer granularity than the one selected by the report slice options.

In [ ]:
var pvReport = ifrs17.PresentValues;
pvReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
pvReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.2"),("LiabilityType", "LIC") }
(await pvReport.ToReportAsync) with {Height = 800}

# Risk Adjustment

In [ ]:
var raReport = ifrs17.RiskAdjustments;
raReport.ColumnSlices = new string[]{};//"GroupOfContract"
raReport.DataFilter = null;//new [] {("GroupOfContract", "DT1.2")};
(await raReport.ToReportAsync) with {Height = 800}

# Actuals

In [ ]:
var writtenActualReport = ifrs17.WrittenActuals;
writtenActualReport.ColumnSlices = new string[]{};//"GroupOfContract"
writtenActualReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await writtenActualReport.ToReportAsync) with {Height = 400}

## Advance, Overdue Actuals

In [ ]:
var accrualActualReport = ifrs17.AccrualActuals;
accrualActualReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
accrualActualReport.DataFilter = null;//new [] {("GroupOfContract", "DT2.1")};
(await accrualActualReport.ToReportAsync) with {Height = 400}

## Deferrable Actuals

In [ ]:
var deferrableActualReport = ifrs17.DeferralActuals;
deferrableActualReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
deferrableActualReport.DataFilter = null;//new [] {("GroupOfContract", "DT1.1")};
(await deferrableActualReport.ToReportAsync) with {Height = 400}

# Fulfilment Cashflow

In [ ]:
var fulfillmentCashflowsReport = ifrs17.FulfillmentCashflows;
fulfillmentCashflowsReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
fulfillmentCashflowsReport.DataFilter = null;// new [] {("GroupOfContract", "DT1.1")};
(await fulfillmentCashflowsReport.ToReportAsync) with {Height = 750}

# Actuarial Experience Adjustment

In [ ]:
var experienceAdjustmentsReport = ifrs17.ExperienceAdjustments;
experienceAdjustmentsReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
experienceAdjustmentsReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await experienceAdjustmentsReport.ToReportAsync) with {Height = 300}

# LRC Technical Margin

In [ ]:
var technicalMarginsReport = ifrs17.TechnicalMargins;
technicalMarginsReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
technicalMarginsReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await technicalMarginsReport.ToReportAsync) with {Height = 600}

# Contractual Service Margin / Loss Component / Loss Recovery Component

In [ ]:
var allocatedTechnicalMarginsReport = ifrs17.AllocatedTechnicalMargins;
allocatedTechnicalMarginsReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
allocatedTechnicalMarginsReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await allocatedTechnicalMarginsReport.ToReportAsync) with {Height = 700}

# LRC Actuarial

In [ ]:
var actuarialLrcReport = ifrs17.ActuarialLrc;
actuarialLrcReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
actuarialLrcReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLrcReport.ToReportAsync) with {Height = 750}

# LRC

In [ ]:
var lrcReport = ifrs17.Lrc;
lrcReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
lrcReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await lrcReport.ToReportAsync) with {Height = 250}

# LIC Actuarial

In [ ]:
var actuarialLicReport = ifrs17.ActuarialLic;
actuarialLicReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
actuarialLicReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await actuarialLicReport.ToReportAsync) with {Height = 750}

# LIC

In [ ]:
var licReport = ifrs17.Lic;
licReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
licReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await licReport.ToReportAsync) with {Height = 250}

# Financial Performance

Use the expand and collapse buttons in the report rows to change the granularity of the figures displayed.

In [ ]:
var financialPerformanceReport = ifrs17.FinancialPerformance;
financialPerformanceReport.ColumnSlices = new string[]{};//"GroupOfContract", "AmountType"
financialPerformanceReport.DataFilter = null; //new [] {("GroupOfContract", "DT1.1")};
(await financialPerformanceReport.ToReportAsync) with { Height = 900, GroupDefaultExpanded = 3 }

In [ ]:
Report.ForDataCube(universe.GetScopes<FinancialPerformance>(identities).Aggregate().FinancialPerformance
                   //.Filter(("GroupOfContract", "DT1.1"))
                  )
    .WithQuerySource(DataSource)
    .SliceRowsBy("VariableType", "EstimateType")
    .SliceColumnsBy(CurrencyGrouper(Args.CurrencyType), "LiabilityType","GroupOfContract")
    .ReportGridOptions(reportHeight: 900, headerColumnWidth: 500, groupDefaultExpanded: 3)
    .ToReport()